# AutoML × ClearML（Notebook Quickstart）

このNotebookは **リポジトリの実行コードには手を入れず**、ユーザーが最小限の入力だけで以下を実行できるようにしたガイドです。

## セル構成
- Setup
- データ登録
- 前処理 + 学習（PipelineController）
- 推論（単一条件入力）
- 推論（最適化 / optimize）

## 使い方
- 各セルには編集ブロック（YAML）があり、ここだけを編集して実行します。
- Notebook用の補助処理は `Notebook_run/` に分離してあります（セル内に長いロジックは置きません）。
- 実行結果（dataset_id / run_id / recommended_model_id など）は `outputs/notebook_state.json` に保存されます。

## 注意
- 推論セルは `input.single` / `input.variables` が空だと実行できません。
  - 学習セルの最後に出る `suggest_single_yaml` / `suggest_optimize_yaml` をコピペして埋めてください。


In [ ]:
# Setup（最初に1回だけ）
import Notebook_run as nb

ctx = nb.bootstrap()
print(f"repo_root: {ctx.repo_root}")
print(f"state_path: {ctx.state_path}")
print(f"config_out_dir: {ctx.config_out_dir}")


In [ ]:
# データ登録（ClearML Dataset 登録）
import Notebook_run as nb

ctx = nb.bootstrap()

# --- 編集ブロック（ここだけ編集）---
# 選択肢メモ:
# - clearml.enabled: true/false
# - clearml.register_raw_dataset: true にすると CSV から ClearML Dataset を作ります
datareg_yaml = """
data:
  csv_path: "data/example.csv"
  target_column: purchase_amount
  test_size: 0.0
  random_seed: 42

clearml:
  enabled: true
  project_name: "AutoML-with-ClearML"
  dataset_project: "AutoML-datasets"
  register_raw_dataset: true
  queue: "default"
  tags: ["automl", "data_registration"]
""".lstrip()

nb.data_registration(ctx, yaml_text=datareg_yaml)


In [ ]:
# 前処理 + 学習（PipelineController）
import Notebook_run as nb

ctx = nb.bootstrap()

# --- 編集ブロック（ここだけ編集）---
# feature_columns:
# - null の場合: target_column 以外を全て feature として扱います
# dataset_id は、直前の「データ登録」結果があれば自動でそちらが優先されます。
pipeline_yaml = """
data:
  dataset_id: ""
  target_column: purchase_amount
  feature_columns: null

clearml:
  project_name: "AutoML-with-ClearML"
  dataset_project: "AutoML-datasets"
""".lstrip()

nb.pipeline_training(ctx, yaml_text=pipeline_yaml)


In [ ]:
# 推論（単一条件入力）
import Notebook_run as nb

ctx = nb.bootstrap()

# --- 編集ブロック（ここだけ編集）---
# model_id は、直前の学習結果 recommended_model_id があれば自動でそちらが優先されます。
# input.single は空だと実行できません（学習セルの suggest_single_yaml をコピペ推奨）。
infer_single_yaml = """
model_id: ""
clearml:
  project_name: "AutoML-with-ClearML"
  task_name: null
input:
  mode: "single"
  single: {}
""".lstrip()

nb.inference_single(ctx, yaml_text=infer_single_yaml)


In [ ]:
# 推論（最適化 / optimize）
import Notebook_run as nb

ctx = nb.bootstrap()

# --- 編集ブロック（ここだけ編集）---
# model_id は、直前の学習結果 recommended_model_id があれば自動でそちらが優先されます。
# input.variables は空だと実行できません（学習セルの suggest_optimize_yaml をコピペ推奨）。
infer_opt_yaml = """
model_id: ""
clearml:
  project_name: "AutoML-with-ClearML"
  task_name: null
input:
  mode: "optimize"
  variables: []
search:
  method: "tpe"
  n_trials: 10
  goal: "max"
  top_k: 10
""".lstrip()

nb.inference_optimize(ctx, yaml_text=infer_opt_yaml)
